In [1]:
import os
import pandas as pd
import ipdb
from datetime import datetime, time

In [2]:
from index import add_xl_to_csv

In [4]:
%ls data
xl_fn = 'data/weekly-2020-02-10 0925.xlsx'
new_df = add_xl_to_csv(xl_fn, '2020.csv')
new_df

weekly-2020-02-10 0925.xlsx


In [ ]:
curr = pd.read_csv('2020.csv')
curr

In [ ]:
res = pd.concat([curr, new_df])
res

In [ ]:
res.drop_duplicates(inplace=True)
res

In [ ]:
res.to_csv('2020.csv', index=False)